# Autograph使用规范

1、被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。

In [2]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [3]:
#np_random每次执行都是一样的结果。
np_random()
np_random()

array([[-0.8533249 , -0.65982598,  0.46978706],
       [-1.19132929, -0.80967162,  0.33337003],
       [ 1.06020708, -0.94404936, -0.1212558 ]])
array([[-0.8533249 , -0.65982598,  0.46978706],
       [-1.19132929, -0.80967162,  0.33337003],
       [ 1.06020708, -0.94404936, -0.1212558 ]])


In [4]:
np.random.randn(3,3)

array([[ 0.25257702,  0.16708006,  0.79210834],
       [-0.34683175,  0.69328068,  1.31706466],
       [ 0.5580037 , -1.50770656, -0.79776019]])

In [5]:
np.random.randn(3,3)

array([[ 1.55126589, -0.80073444, -0.92080275],
       [-0.50392245, -0.69853033,  0.21980977],
       [-0.1379339 ,  0.89195606,  1.2976263 ]])

In [6]:
tf_random()
tf_random()

[[0.9276793 -0.528112411 1.88184559]
 [-1.20099926 -1.04864311 0.245232761]
 [-1.71584272 -1.03118122 -0.0328678861]]
[[-0.0503324643 -0.459995121 -1.84465837]
 [1.13207293 0.68550396 -0.458074749]
 [1.17981255 1.2004118 -0.352737874]]


2、避免在@tf.function修饰的函数内部定义tf.Variable.

In [7]:

x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()

2
3


<tf.Tensor: id=32, shape=(), dtype=float32, numpy=3.0>

In [ ]:
#报错
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)
inner_var()

3、被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。

In [11]:
tensor_list = []

#@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: id=81, shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: id=82, shape=(), dtype=float32, numpy=6.0>]


In [12]:
tensor_list = []

@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
